## Question 1

### Problématique du Challenge

SNCF-Transilien, opérateur de trains de banlieue en Île-de-France, souhaite mieux anticiper l'augmentation du nombre de validations quotidiennes des cartes à puce de ses voyageurs dans les gares. Cette prévision à moyen-long terme permettra de proposer des services plus adaptés et d'améliorer la performance de l'exploitation. Le but est donc de prédire le nombre de validations par jour et par gare pour mieux planifier les ressources et les infrastructures nécessaires.

### Objectif

Le but de ce challenge est de prédire le nombre de validations par jour pour chaque gare entre le 1er janvier 2023 et le 30 juin 2023. La précision de ces prévisions sera mesurée en utilisant le MAPE (Mean Absolute Percentage Error).

### Approche pour Répondre à la Problématique

1. **Exploration des Données** :
   - **Analyse Exploratoire des Données (EDA)** : Visualiser et analyser les tendances et les schémas dans les données historiques (train.csv).
   - **Identification des Saisonnalités et Tendances** : Déterminer si les données présentent des schémas saisonniers, hebdomadaires ou annuels.

2. **Prétraitement des Données** :
   - **Nettoyage des Données** : Gérer les valeurs manquantes, les anomalies et les incohérences.
   - **Feature Engineering** : Créer de nouvelles variables explicatives (features) qui pourraient améliorer la performance du modèle, par exemple, les lags, les moyennes mobiles, etc.
   - **Encodage des Variables Catégorielles** : Convertir les variables catégorielles en variables numériques.

3. **Modélisation** :
   - **Sélection des Modèles** : Tester différents modèles de machine learning appropriés pour les séries temporelles, comme les modèles ARIMA, les régressions linéaires, les arbres de décision, les modèles basés sur les forêts aléatoires, et les modèles de réseaux de neurones (RNN, LSTM).
   - **Entraînement des Modèles** : Entraîner les modèles sur les données de formation (train.csv) en utilisant les variables explicatives (x_train) et la variable cible (y_train).

4. **Évaluation et Validation** :
   - **Validation Croisée** : Utiliser des techniques de validation croisée pour évaluer les performances des modèles et éviter le surapprentissage (overfitting).
   - **Calcul du MAPE** : Calculer la métrique MAPE sur les données de test (x_test) pour mesurer la précision des prédictions.

5. **Prédiction et Soumission** :
   - **Prédiction** : Utiliser le modèle sélectionné pour prédire les validations pour chaque gare sur la période de test (janvier à juin 2023).
   - **Soumission des Résultats** : Préparer le fichier de soumission avec les prédictions.
   
   


## Question 2

Notre base de données est bien décomposée en un échantillon de train (d’entrainement) et de test. Nous décomposons ci-dessous la base de train en une base pour l’estimation et une autre pour la validation.

In [2]:
import pandas as pd

# Charger les fichiers CSV
x_train = pd.read_csv('x_train_data.csv')
y_train = pd.read_csv('y_train_data.csv')
x_test = pd.read_csv('x_test_data.csv')

# Fusionner les x_train et y_train pour une séparation chronologique
train_data = pd.merge(x_train, y_train, left_index=True, right_index=True)

# Trier les données par date
train_data['date'] = pd.to_datetime(train_data['date'])
train_data = train_data.sort_values(by='date')

x_test['date'] = pd.to_datetime(x_test['date'])

# Définir une date de séparation (par exemple, 80% des données pour l'estimation)
split_date = train_data['date'].quantile(0.8)

# Séparer les données en ensembles d'estimation et de validation
train_set = train_data[train_data['date'] <= split_date]
validation_set = train_data[train_data['date'] > split_date]

# Séparer les variables explicatives et la variable cible pour chaque ensemble
X_train = train_set.drop(columns=['y'])
y_train = train_set['y']
X_val = validation_set.drop(columns=['y'])
y_val = validation_set['y']

# Sauvegarder les ensembles réduits si nécessaire
X_train.to_csv('X_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
X_val.to_csv('X_val.csv', index=False)
y_val.to_csv('y_val.csv', index=False)

# Vérifier les tailles des ensembles réduits
print("Tailles des ensembles d'estimation et de validation :")
print("X_train :", X_train.shape)
print("y_train :", y_train.shape)
print("X_val :", X_val.shape)
print("y_val :", y_val.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'x_train_data.csv'

## Question 3

### Description et Statistiques des Features

#### Jour de la Semaine (`day_of_week`) :
- **Description** : Représente les jours de la semaine de 0 (lundi) à 6 (dimanche).
- **Statistiques** :
  - Moyenne
  - Écart-type
  - Distribution par jour de la semaine

#### Mois (`month`) :
- **Description** : Représente le mois de l'année de 1 (janvier) à 12 (décembre).
- **Statistiques** :
  - Moyenne
  - Écart-type
  - Distribution par mois

#### Année (`year`) :
- **Description** : Représente l'année des données.
- **Statistiques** :
  - Distribution par année

#### Week-end (`weekend`) :
- **Description** : Indicateur binaire (1 pour samedi et dimanche, 0 pour le reste de la semaine).
- **Statistiques** :
  - Proportion de jours de week-end

#### Jour avant un jour férié (`day_before_holiday`) :
- **Description** : Indicateur binaire (1 si le jour suivant est férié, 0 sinon).
- **Statistiques** :
  - Proportion de jours avant un jour férié

#### Jour après un jour férié (`day_after_holiday`) :
- **Description** : Indicateur binaire (1 si le jour précédent est férié, 0 sinon).
- **Statistiques** :
  - Proportion de jours après un jour férié


In [16]:
# Fonction pour créer les nouvelles features
def create_features(df):
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    df['day_before_holiday'] = df['ferie'].shift(-1, fill_value=0)
    df['day_after_holiday'] = df['ferie'].shift(1, fill_value=0)
    return df

# Appliquer la fonction pour créer les features
X_train = create_features(X_train)
X_val = create_features(X_val)
x_test = create_features(x_test)

# Calcul des statistiques pour les features dans X_train
day_of_week_stats = {
    'mean': X_train['day_of_week'].mean(),
    'std': X_train['day_of_week'].std(),
    'distribution': X_train['day_of_week'].value_counts().to_dict()
}

month_stats = {
    'mean': X_train['month'].mean(),
    'std': X_train['month'].std(),
    'distribution': X_train['month'].value_counts().to_dict()
}

year_stats = {
    'distribution': X_train['year'].value_counts().to_dict()
}

weekend_stats = {
    'proportion': X_train['weekend'].mean()
}

day_before_holiday_stats = {
    'proportion': X_train['day_before_holiday'].mean()
}

day_after_holiday_stats = {
    'proportion': X_train['day_after_holiday'].mean()
}

# Création d'un dataframe pour afficher les statistiques
stats_df = pd.DataFrame({
    'Feature': ['day_of_week', 'month', 'year', 'weekend', 'day_before_holiday', 'day_after_holiday'],
    'Mean/Proportion': [
        day_of_week_stats['mean'], 
        month_stats['mean'], 
        np.nan, 
        weekend_stats['proportion'], 
        day_before_holiday_stats['proportion'], 
        day_after_holiday_stats['proportion']
    ],
    'Std': [
        day_of_week_stats['std'], 
        month_stats['std'], 
        np.nan, 
        np.nan, 
        np.nan, 
        np.nan
    ],
    'Distribution': [
        day_of_week_stats['distribution'], 
        month_stats['distribution'], 
        year_stats['distribution'], 
        np.nan, 
        np.nan, 
        np.nan
    ]
})

# Afficher le tableau des statistiques
(stats_df)

,Feature,Mean/Proportion,Std,Distribution
0,day_of_week,2.990070,1.997629,"{0: 141434, 3: 141327, 4: 141227, 1: 140938, 2..."
1,month,6.292195,3.467670,"{1: 91907, 5: 91368, 3: 91136, 4: 86876, 2: 84..."
2,year,NaN,NaN,"{2017: 156498, 2016: 155516, 2018: 154717, 201..."
3,weekend,0.282968,NaN,NaN
4,day_before_holiday,0.030459,NaN,NaN
5,day_after_holiday,0.030460,NaN,NaN


In [8]:
train_set

,date,station,job,ferie,vacances,index,y,day_of_week,month,year,weekend,day_before_holiday,day_after_holiday
0,2015-01-01,1J7,1,1,1,2015-01-01_1J7,7,3,1,2015,0,1,0
288,2015-01-01,S3E,1,1,1,2015-01-01_S3E,103,3,1,2015,0,1,1
287,2015-01-01,RDK,1,1,1,2015-01-01_RDK,98,3,1,2015,0,1,1
286,2015-01-01,I9U,1,1,1,2015-01-01_I9U,277,3,1,2015,0,1,1
285,2015-01-01,OM7,1,1,1,2015-01-01_OM7,398,3,1,2015,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215714,2021-05-31,SYW,1,0,0,2021-05-31_SYW,1885,0,5,2021,0,0,0
1215713,2021-05-31,003,1,0,0,2021-05-31_003,417,0,5,2021,0,0,0
1215712,2021-05-31,EMR,1,0,0,2021-05-31_EMR,1204,0,5,2021,0,0,0
1215711,2021-05-31,5KJ,1,0,0,2021-05-31_5KJ,4810,0,5,2021,0,0,0


In [9]:
validation_set

,date,station,job,ferie,vacances,index,y,day_of_week,month,year,weekend,day_before_holiday,day_after_holiday
19379,2021-06-01,B3W,1,0,0,2021-06-01_B3W,838,1,6,2021,0,0,0
19540,2021-06-01,DRI,1,0,0,2021-06-01_DRI,144,1,6,2021,0,0,0
19378,2021-06-01,VGQ,1,0,0,2021-06-01_VGQ,551,1,6,2021,0,0,0
19377,2021-06-01,BNR,1,0,0,2021-06-01_BNR,168,1,6,2021,0,0,0
19376,2021-06-01,3KU,1,0,0,2021-06-01_3KU,2219,1,6,2021,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229565,2022-12-31,B65,0,0,1,2022-12-31_B65,1161,5,12,2022,1,0,0
1229564,2022-12-31,2P1,0,0,1,2022-12-31_2P1,2118,5,12,2022,1,0,0
1229563,2022-12-31,D68,0,0,1,2022-12-31_D68,673,5,12,2022,1,0,0
1229574,2022-12-31,KQC,0,0,1,2022-12-31_KQC,3128,5,12,2022,1,0,0


In [13]:
x_test_set

,index,date,station,job,ferie,vacances,day_of_week,month,year,weekend,day_before_holiday,day_after_holiday
0,2023-01-01_1J7,2023-01-01,1J7,0,1,1,6,1,2023,1,1,0
1,2023-01-01_O2O,2023-01-01,O2O,0,1,1,6,1,2023,1,1,1
2,2023-01-01_8QR,2023-01-01,8QR,0,1,1,6,1,2023,1,1,1
3,2023-01-01_L58,2023-01-01,L58,0,1,1,6,1,2023,1,1,1
4,2023-01-01_UMC,2023-01-01,UMC,0,1,1,6,1,2023,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
78647,2023-05-31_N9K,2023-05-31,N9K,1,0,0,2,5,2023,0,0,0
78648,2023-05-31_P6E,2023-05-31,P6E,1,0,0,2,5,2023,0,0,0
78649,2023-05-31_BDC,2023-05-31,BDC,1,0,0,2,5,2023,0,0,0
78650,2023-05-31_QD6,2023-05-31,QD6,1,0,0,2,5,2023,0,0,0


In [14]:
# Calculer les statistiques pour les features dans X_train
import numpy as np

# Calcul des statistiques
day_of_week_stats = {
    'mean': X_train['day_of_week'].mean(),
    'std': X_train['day_of_week'].std(),
    'distribution': X_train['day_of_week'].value_counts().to_dict()
}

month_stats = {
    'mean': X_train['month'].mean(),
    'std': X_train['month'].std(),
    'distribution': X_train['month'].value_counts().to_dict()
}

year_stats = {
    'distribution': X_train['year'].value_counts().to_dict()
}

weekend_stats = {
    'proportion': X_train['weekend'].mean()
}

day_before_holiday_stats = {
    'proportion': X_train['day_before_holiday'].mean()
}

day_after_holiday_stats = {
    'proportion': X_train['day_after_holiday'].mean()
}

# Création d'un dataframe pour afficher les statistiques
stats_df = pd.DataFrame({
    'Feature': ['day_of_week', 'month', 'year', 'weekend', 'day_before_holiday', 'day_after_holiday'],
    'Mean/Proportion': [
        day_of_week_stats['mean'], 
        month_stats['mean'], 
        np.nan, 
        weekend_stats['proportion'], 
        day_before_holiday_stats['proportion'], 
        day_after_holiday_stats['proportion']
    ],
    'Std': [
        day_of_week_stats['std'], 
        month_stats['std'], 
        np.nan, 
        np.nan, 
        np.nan, 
        np.nan
    ],
    'Distribution': [
        day_of_week_stats['distribution'], 
        month_stats['distribution'], 
        year_stats['distribution'], 
        np.nan, 
        np.nan, 
        np.nan
    ]
})

import ace_tools as tools; tools.display_dataframe_to_user(name="Feature Statistics", dataframe=stats_df)

stats_df


KeyError: 'day_of_week'